In [ ]:
# |default_exp mongo_episode

In [ ]:
from mongo import print_logs

print_logs(5)

{'_id': ObjectId('6786a219ff4d877f74aaf512'), 'operation': 'update', 'entite': 'episodes', 'desc': '22 Dec 2024 10:59 - Les nouvelles pages de Marc Dugain, Emmanuelle Lambert, Emil Ferris, Fabrice Caro et Mathieu Palain', 'date': datetime.datetime(2025, 1, 14, 18, 42, 49, 496000)}
{'_id': ObjectId('6786a210ff4d877f74aaf50c'), 'operation': 'insert', 'entite': 'episodes', 'desc': '12 Jan 2025 10:59 - Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...', 'date': datetime.datetime(2025, 1, 14, 18, 42, 40, 679000)}
{'_id': ObjectId('6786a1d1de806a34f3f18d03'), 'operation': 'update', 'entite': 'episodes', 'desc': '22 Dec 2024 10:59 - Les nouvelles pages de Marc Dugain, Emmanuelle Lambert, Emil Ferris, Fabrice Caro et Mathieu Palain', 'date': datetime.datetime(2025, 1, 14, 18, 41, 37, 722000)}
{'_id': ObjectId('6786a1c5de806a34f3f18cfd'), 'operation': 'insert', 'entite': 'episodes', 'desc': '12 Jan 2025 10:59 - Les nouvelles pages de Vanessa Springora, Ha


# Episode entity

In [ ]:
# |export

import os
from git import Repo

AUDIO_PATH = "audios"


def get_audio_path(audio_path=AUDIO_PATH, year: str = "2024"):
    """
    audio_path: str
        relative path to audio files
    will add year as subdirectory
    return full audio path and create dir if it doesn t exist
    """

    def get_git_root(path):
        git_repo = Repo(path, search_parent_directories=True)
        return git_repo.git.rev_parse("--show-toplevel")

    project_root = get_git_root(os.getcwd())
    full_audio_path = os.path.join(project_root, audio_path, year)

    # create dir if it doesn t exist
    if not os.path.exists(full_audio_path):
        os.makedirs(full_audio_path)

    return full_audio_path

In [ ]:
# |export

from bson import ObjectId
from mongo import get_collection, get_DB_VARS, mongolog
from datetime import datetime
import requests

DATE_FORMAT = "%Y-%m-%dT%H:%M:%S.%f%z"
LOG_DATE_FORMAT = "%d %b %Y %H:%M"


class Episode:
    def __init__(self, date: str, titre: str, collection_name: str = "episodes"):
        """
        Episode is a class that represents a generic Episode entity in the database.
        :param date: The date for this episode at the format "2024-12-22T09:59:39.000+00:00" parsed by "%Y-%m-%dT%H:%M:%S.%f%z".
        :param titre: The title of this episode.
        :param collection_name: The name of the collection. default: "episodes".
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        self.date = Episode.get_date_from_string(date)
        self.titre = titre
        self.description = None
        self.url_telechargement = None
        self.audio_rel_filename = None
        self.transcription = None
        self.type = None
        self.duree = -1  # in seconds

    def exists(self) -> bool:
        """
        Check if the episode exists in the database.
        :return: True if the episode exists, False otherwise.
        """
        return (
            self.collection.find_one({"titre": self.titre, "date": self.date})
            is not None
        )

    def keep(self) -> int:
        """
        download the audio file if needed
        Keep the episode in the database

        retourne 1 si 1 entree est creee en base
        0 sinon
        """
        message_log = f"{Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} - {self.titre}"
        if not self.exists():
            print(
                f"Episode du {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} nouveau: Duree: {self.duree}, Type: {self.type}"
            )
            mongolog("insert", self.collection.name, message_log)
            self.download_audio(verbose=True)
            self.collection.insert_one(
                {
                    "titre": self.titre,
                    "date": self.date,
                    "description": self.description,
                    "url": self.url_telechargement,
                    "audio_rel_filename": self.audio_rel_filename,
                    "transcription": self.transcription,
                    "type": self.type,
                    "duree": self.duree,
                }
            )
            return 1
        else:
            print(
                f"Episode du {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} deja existant"
            )
            mongolog("update", self.collection.name, message_log)
            return 0

    def remove(self):
        """
        Remove the episode from the database.
        """
        message_log = f"{Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} - {self.titre}"
        self.collection.delete_one({"titre": self.titre, "date": self.date})
        mongolog("delete", self.collection.name, message_log)

    def get_oid(self) -> ObjectId:
        """
        Get the object id of the episode.
        :return: The object id of the episode. (bson.ObjectId)
        None if does not exist.
        """
        document = self.collection.find_one({"titre": self.titre, "date": self.date})
        if document:
            return document["_id"]
        else:
            return None

    @staticmethod
    def get_date_from_string(date: str) -> datetime:
        """
        Get the datetime object from a string.
        :param date: The date string.
        :return: The datetime object.
        """
        return datetime.strptime(date, DATE_FORMAT)

    @staticmethod
    def get_string_from_date(date: datetime, format: str = None) -> str:
        """
        Get the string from a datetime object.
        :param date: The datetime object.
        :param format: The format of the string. default: None and DATE_FORMAT will be used.
        :return: The date string.
        """
        if format is not None:
            return date.strftime(format)
        else:
            return date.strftime(DATE_FORMAT)

    def __str__(self):
        return f"""
        Date: {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)}
        Titre: {self.titre}
        Description: {self.description}
        URL de téléchargement: {self.url_telechargement}
        Fichier audio: {self.audio_rel_filename}
        Duree: {self.duree} en secondes 
        """

    def __repr__(self):
        return self.__str__()

    def download_audio(self, verbose=False):
        """
        based on url_telechargement
        will download audio file and store in AUDIO_PATH/year
        """
        if self.url_telechargement is None:
            return
        year = str(self.date.year)
        full_audio_path = get_audio_path(AUDIO_PATH, year)
        full_filename = os.path.join(
            full_audio_path, os.path.basename(self.url_telechargement)
        )
        self.audio_rel_filename = os.path.relpath(
            full_filename, get_audio_path(AUDIO_PATH, year="")
        )
        # Vérification si le fichier existe déjà
        if not os.path.exists(full_filename):
            if verbose:
                print(
                    f"Téléchargement de {self.url_telechargement} vers {full_filename}"
                )
            response = requests.get(self.url_telechargement)
            with open(full_filename, "wb") as file:
                file.write(response.content)
        else:
            if verbose:
                print(f"Le fichier {full_filename} existe déjà. Ignoré.")

# RSS_episode

In [ ]:
# |export

from feedparser.util import FeedParserDict
from transformers import pipeline

RSS_DUREE_MINI_MINUTES = 15
RSS_DATE_FORMAT = "%a, %d %b %Y %H:%M:%S %z"  # "Sun, 29 Dec 2024 10:59:39 +0100"


class RSS_episode(Episode):
    def __init__(self, date: str, titre: str):
        """
        RSS_episode is a class that represents an RSS episode in the database episodes.
        :param date: The date for this episode at the format "2024-12-22T09:59:39.000+00:00" parsed by "%Y-%m-%dT%H:%M:%S.%f%z".
        :param titre: The title of this episode.
        """
        super().__init__(date, titre)

    @classmethod
    def from_feed_entry(cls, feed_entry: FeedParserDict) -> "RSS_episode":
        """
        Create an RSS episode from a feed entry.
        :param feed_entry: The feed entry.
        :return: The RSS episode.
        """

        date_rss = datetime.strptime(feed_entry.published, RSS_DATE_FORMAT)
        date_rss_str = cls.get_string_from_date(date_rss, DATE_FORMAT)
        inst = cls(
            date=date_rss_str,
            titre=feed_entry.title,
        )
        inst.description = feed_entry.summary

        for link in feed_entry.links:
            if link.type == "audio/mpeg":
                inst.url_telechargement = link.href
                break

        # self.audio_rel_filename = None
        # self.transcription = None
        inst.type = cls.set_titre(inst.titre + " " + inst.description)
        inst.duree = cls.get_duree_in_seconds(feed_entry.itunes_duration)  # in seconds

        return inst

    @staticmethod
    def get_duree_in_seconds(duree: str) -> int:
        """
        Get the duration in seconds from a string.
        :param duree: The duration string at the format "HH:MM:SS" or "HH:MM".
        :return: The duration in seconds.
        """
        duree = duree.split(":")
        if len(duree) == 3:
            return int(duree[0]) * 3600 + int(duree[1]) * 60 + int(duree[2])
        elif len(duree) == 2:
            return int(duree[0]) * 60 + int(duree[1])
        else:
            return int(duree[0])

    def keep(self) -> int:
        """
        Keep the episode in the database.
        only if duration > RSS_DUREE_MINI_MINUTES * 60
        only if type == livres

        retourne 1 si 1 entree est creee en base
        0 sinon
        """
        if (self.duree > RSS_DUREE_MINI_MINUTES * 60) & (self.type == "livres"):
            return super().keep()
        else:
            print(
                f"Episode du {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} ignored: Duree: {self.duree}, Type: {self.type}"
            )
            return 0

    @staticmethod
    def set_titre(description: str) -> str:
        """
        use bart meta model from huggingface to classify episodes from
        ["livres", "films", "pièces de théâtre"]
        """
        # Charger le pipeline de classification de texte
        classifier = pipeline(
            "zero-shot-classification", model="facebook/bart-large-mnli"
        )
        # Labels possibles
        labels = ["livres", "films", "pièces de théâtre"]

        result = classifier(description, labels)
        return result["labels"][0]

In [ ]:
from datetime import datetime
import pytz

now = datetime.now(tz=pytz.timezone("Europe/Paris"))

rss1 = RSS_episode(RSS_episode.get_string_from_date(now), "test RSS 1")
print(f"Est-ce que l episode existe ? {rss1.exists()}")

rss1.duree = 4000
rss1.type = "livres"
rss1.keep()
print(f"et maintenant, st-ce que rss1 existe ? {rss1.exists()}")
print(f"et voici l'id de rss1 : {rss1.get_oid()}")

rss1.remove()
print(f"après nettoyage, est-ce que rss1 existe ? {rss1.exists()}")
print(f"et son oid : {rss1.get_oid()}")

Est-ce que l episode existe ? False
Episode du 14 Jan 2025 18:45 nouveau: Duree: 4000, Type: livres
et maintenant, st-ce que rss1 existe ? True
et voici l'id de rss1 : 6786a2c1f4481d81a88185e4
après nettoyage, est-ce que rss1 existe ? False
et son oid : None


In [ ]:
import feedparser
from rss import get_RSS_URL

parsed_flow = feedparser.parse(get_RSS_URL())

rss2 = RSS_episode.from_feed_entry(parsed_flow.entries[0])
rss2

Device set to use cpu



        Date: 12 Jan 2025 10:59
        Titre: Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...
        Description: durée : 00:48:41 - Le Masque et la Plume - par : Rebecca Manzoni - Un passé nazi qui refait surface ; une cité magique abrite un amour perdu ; un cinéaste entre un tournage en Afrique et la chute d'un homme nu ; un danseur offre à son fils adoptif la quête d'un héritage familial mouvementé ; un jeune père se confronte à son propre passé douloureux. - invités : Raphaelle Leyris, Hubert ARTUS, Jean-Marc Proust, Elisabeth Philippe - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Hubert Artus : Journaliste et chroniqueur littéraire, Jean-Marc Proust : Auteur et critique (Slate), Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
        URL de téléchargement: https://rf.proxycast.org/ad97aa2e-ebfc-4d00-8739-4ca72192e726/14007-12.01.2025-ITEMA_23993269-2025F4007S0012-22.mp3
        Fichie

In [ ]:
rss2.keep()
rss2.exists()

Episode du 12 Jan 2025 10:59 deja existant


True

In [ ]:
# rss2.remove()

In [ ]:
print_logs(5)

{'_id': ObjectId('6786a2c6f4481d81a88185e9'), 'operation': 'update', 'entite': 'episodes', 'desc': '12 Jan 2025 10:59 - Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...', 'date': datetime.datetime(2025, 1, 14, 18, 45, 42, 384000)}
{'_id': ObjectId('6786a2c1f4481d81a88185e6'), 'operation': 'delete', 'entite': 'episodes', 'desc': '14 Jan 2025 18:45 - test RSS 1', 'date': datetime.datetime(2025, 1, 14, 18, 45, 37, 158000)}
{'_id': ObjectId('6786a2c1f4481d81a88185e3'), 'operation': 'insert', 'entite': 'episodes', 'desc': '14 Jan 2025 18:45 - test RSS 1', 'date': datetime.datetime(2025, 1, 14, 18, 45, 37, 145000)}
{'_id': ObjectId('6786a219ff4d877f74aaf512'), 'operation': 'update', 'entite': 'episodes', 'desc': '22 Dec 2024 10:59 - Les nouvelles pages de Marc Dugain, Emmanuelle Lambert, Emil Ferris, Fabrice Caro et Mathieu Palain', 'date': datetime.datetime(2025, 1, 14, 18, 42, 49, 496000)}
{'_id': ObjectId('6786a210ff4d877f74aaf50c'), 'operation': '

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("py mongo helper episodes.ipynb", ".")